In [1]:
import nbimporter
from utils import *

In [2]:
class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size, learning_rate=0.01, epochs=500):
        print("Initialisation du NeuralNetwork...")
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.learning_rate = learning_rate
        self.epochs = epochs

        self.weights_input_hidden = np.random.randn(self.input_size, self.hidden_size) * 0.1
        self.bias_hidden = np.zeros((1, self.hidden_size))
        self.weights_hidden_output = np.random.randn(self.hidden_size, self.output_size) * 0.1
        self.bias_output = np.zeros((1, self.output_size))

    def softmax(self, z):
        exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))
        return exp_z / np.sum(exp_z, axis=1, keepdims=True)

    def softmax_derivative(self, z):
        return z * (1 - z)

    def fit(self, X, y, batch_size=32):
        print("Début de l'entraînement du NeuralNetwork...")
        y_one_hot = np.eye(self.output_size)[y]

        for epoch in range(self.epochs):
            if epoch % 50 == 0:
                print(f"Époque {epoch}/{self.epochs}")
            for i in range(0, X.shape[0], batch_size):
                X_batch = X[i:i + batch_size]
                y_batch = y_one_hot[i:i + batch_size]

                hidden_layer_activation = np.dot(X_batch, self.weights_input_hidden) + self.bias_hidden
                hidden_layer_output = self.softmax(hidden_layer_activation)
                final_layer_activation = np.dot(hidden_layer_output, self.weights_hidden_output) + self.bias_output
                output = self.softmax(final_layer_activation)

                error = y_batch - output
                d_output = error

                d_hidden_layer = np.dot(d_output, self.weights_hidden_output.T) * self.softmax_derivative(hidden_layer_output)

                self.weights_hidden_output += np.dot(hidden_layer_output.T, d_output) * self.learning_rate
                self.bias_output += np.sum(d_output, axis=0, keepdims=True) * self.learning_rate
                self.weights_input_hidden += np.dot(X_batch.T, d_hidden_layer) * self.learning_rate
                self.bias_hidden += np.sum(d_hidden_layer, axis=0, keepdims=True)

    def predict(self, X):
        hidden_layer_activation = np.dot(X, self.weights_input_hidden) + self.bias_hidden
        hidden_layer_output = self.softmax(hidden_layer_activation)
        final_layer_activation = np.dot(hidden_layer_output, self.weights_hidden_output) + self.bias_output
        output = self.softmax(final_layer_activation)
        return np.argmax(output, axis=1)

class LogisticRegression:
    def __init__(self, learning_rate=0.01, epochs=500):
        print("Initialisation de la LogisticRegression...")
        self.learning_rate = learning_rate
        self.epochs = epochs

    def fit(self, X, y):
        print("Début de l'entraînement de la LogisticRegression...")
        self.weights = np.zeros((X.shape[1], len(np.unique(y))))
        self.bias = np.zeros((1, len(np.unique(y))))
        m = len(y)
        y_one_hot = np.eye(len(np.unique(y)))[y]

        for epoch in range(self.epochs):
            if epoch % 50 == 0:
                print(f"Époque {epoch}/{self.epochs}")
            linear_model = np.dot(X, self.weights) + self.bias
            predictions = self.softmax(linear_model)

            dw = (1 / m) * np.dot(X.T, (predictions - y_one_hot))
            db = (1 / m) * np.sum(predictions - y_one_hot, axis=0, keepdims=True)

            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db

    def softmax(self, z):
        exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))
        return exp_z / np.sum(exp_z, axis=1, keepdims=True)

    def predict(self, X):
        linear_model = np.dot(X, self.weights) + self.bias
        predictions = self.softmax(linear_model)
        return np.argmax(predictions, axis=1)

class XGBoost:
    def __init__(self, learning_rate=0.1, n_estimators=100, max_depth=3):
        print("Initialisation de XGBoost...")
        self.learning_rate = learning_rate
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.models = []

    def fit(self, X, y):
        print("Début de l'entraînement de XGBoost...")
        y_one_hot = np.eye(len(np.unique(y)))[y]
        residual = y_one_hot.copy()

        for i in range(self.n_estimators):
            model = LogisticRegression(learning_rate=self.learning_rate, epochs=100)
            model.fit(X, np.argmax(residual, axis=1))
            predictions = model.predict(X)
            predictions_one_hot = np.eye(len(np.unique(y)))[predictions]
            residual -= predictions_one_hot
            self.models.append(model)

    def predict(self, X):
        predictions = np.zeros((X.shape[0], len(self.models[0].weights[0])))
        for model in self.models:
            pred = model.predict(X)
            predictions += np.eye(len(predictions[0]))[pred]
        return np.argmax(predictions, axis=1)


In [3]:
def manual_hyperparameter_search(X, y):
    print("Début de la recherche manuelle des hyperparamètres...")

    # Liste des classes uniques
    unique_classes = np.unique(y)
    num_classes = len(unique_classes)

    # Création d'un mapping entre étiquettes réelles et indices
    class_to_index = {cls: idx for idx, cls in enumerate(unique_classes)}
    index_to_class = {idx: cls for cls, idx in class_to_index.items()}

    # Conversion des étiquettes en indices pour one-hot encoding
    y_indices = np.array([class_to_index[label] for label in y])

    # Division des données
    X_train, X_val, y_train, y_val = train_test_split(X, y_indices, test_size=0.2, random_state=42)

    print(f"Train size: {len(X_train)}, Validation size: {len(X_val)}")

    best_f1 = 0
    best_model = None
    best_conf_matrix = None

    # Recherche pour les réseaux de neurones
    for learning_rate in [0.01, 0.05, 0.1]:
        for epochs in [50, 100, 150]:
            for hidden_size in [10, 20, 30]:
                model = NeuralNetwork(
                    input_size=X_train.shape[1],
                    hidden_size=hidden_size,
                    output_size=num_classes,
                    learning_rate=learning_rate,
                    epochs=epochs,
                )
                model.fit(X_train, y_train)
                f1, conf_matrix = evaluate_model_with_mapping(model, X_val, y_val, index_to_class)

                print(f"Testé: lr={learning_rate}, epochs={epochs}, hidden_size={hidden_size} => F1={f1}")

                if f1 > best_f1:
                    best_f1 = f1
                    best_model = model
                    best_conf_matrix = conf_matrix

    # Recherche pour XGBoost
    for learning_rate in [0.1, 0.2]:
        for n_estimators in [50, 100]:
            model = XGBoost(learning_rate=learning_rate, n_estimators=n_estimators, max_depth=3)
            model.fit(X_train, y_train)
            f1, conf_matrix = evaluate_model_with_mapping(model, X_val, y_val, index_to_class)

            print(f"Testé: lr={learning_rate}, n_estimators={n_estimators} => F1={f1}")

            if f1 > best_f1:
                best_f1 = f1
                best_model = model
                best_conf_matrix = conf_matrix

    print("Recherche manuelle terminée.")
    print("Meilleur F1:", best_f1)
    print("Matrice de confusion finale :\n", best_conf_matrix)

    return best_model, best_f1, best_conf_matrix